This module is for __HOURLY OHLCV__ data, set at __48__ hours:

    ** 1. Merge the pumped signals with time stamps to get vertical format for voting
    ** 2. Append the output whethere the coin was pumped or not (binary 0)

In [2]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
import pickle
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import the pumped data

### Step 1. Create the timestamp for the first hours 

In [3]:
p = pd.read_pickle("../pump_data_with_social.pkl")
p = p[['channelLink','channelTitle','currency','exchange','signalTime','pumptime_edited','timestamp']]
p.head(2)

,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp
0,https://t.me/crypto_pump_island,Crypto Pump Island,ADX,Binance,2019-07-26T17:59:59Z,2019-07-26 17:59:59+00:00,1564163999
1,https://t.me/SignalsKingdom,Signal Kingdom,ADX,Binance,2019-07-05T17:00:32,2019-07-05 17:00:32+00:00,1562346032


Ther are .... unique points in time...

In [5]:
#Number of unique time points
len(pd.unique(p['timestamp']))

422

In [6]:
#Number of unique coins points
len(pd.unique(p['currency']))

154

In [7]:
3600*24

86400

In [8]:
1562346032 - (1562346032 % (86400))

1562284800

..Strange beheavior, must write each function one by one...

In [9]:
def getHourstamp(dfo: pd.DataFrame, num_hours: int=1, var:str='timestamp'):
    '''
    Append a column that contains the timestamp at 12:00 AM (0:00) of the
    original var variable.
    :param df: dataframe in question 
    :param num_date: number of dates to go back to
    :param var str: name of the variable that has original timestamp
    '''
    df = dfo.copy()
    df[var+'_int'] = df[var].apply(int)
    for d in range(num_hours+1):
        df[var+'_h'+str(d)] = df[var+'_int'].apply(lambda t : t - (t % (3600*24)) - d*(3600*24))
    df = df.drop(columns=var+'_int')
    return df

In [10]:
num_hours = 48
p = getHourstamp(p,num_hours)
p.head()

,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp,timestamp_h0,timestamp_h1,timestamp_h2,timestamp_h3,timestamp_h4,timestamp_h5,timestamp_h6,timestamp_h7,timestamp_h8,timestamp_h9,timestamp_h10,timestamp_h11,timestamp_h12,timestamp_h13,timestamp_h14,timestamp_h15,timestamp_h16,timestamp_h17,timestamp_h18,timestamp_h19,timestamp_h20,timestamp_h21,timestamp_h22,timestamp_h23,timestamp_h24,timestamp_h25,timestamp_h26,timestamp_h27,timestamp_h28,timestamp_h29,timestamp_h30,timestamp_h31,timestamp_h32,timestamp_h33,timestamp_h34,timestamp_h35,timestamp_h36,timestamp_h37,timestamp_h38,timestamp_h39,timestamp_h40,timestamp_h41,timestamp_h42,timestamp_h43,timestamp_h44,timestamp_h45,timestamp_h46,timestamp_h47,timestamp_h48
0,https://t.me/crypto_pump_island,Crypto Pump Island,ADX,Binance,2019-07-26T17:59:59Z,2019-07-26 17:59:59+00:00,1564163999,1564099200,1564012800,1563926400,1563840000,1563753600,1563667200,1563580800,1563494400,1563408000,1563321600,1563235200,1563148800,1563062400,1562976000,1562889600,1562803200,1562716800,1562630400,1562544000,1562457600,1562371200,1562284800,1562198400,1562112000,1562025600,1561939200,1561852800,1561766400,1561680000,1561593600,1561507200,1561420800,1561334400,1561248000,1561161600,1561075200,1560988800,1560902400,1560816000,1560729600,1560643200,1560556800,1560470400,1560384000,1560297600,1560211200,1560124800,1560038400,1559952000
1,https://t.me/SignalsKingdom,Signal Kingdom,ADX,Binance,2019-07-05T17:00:32,2019-07-05 17:00:32+00:00,1562346032,1562284800,1562198400,1562112000,1562025600,1561939200,1561852800,1561766400,1561680000,1561593600,1561507200,1561420800,1561334400,1561248000,1561161600,1561075200,1560988800,1560902400,1560816000,1560729600,1560643200,1560556800,1560470400,1560384000,1560297600,1560211200,1560124800,1560038400,1559952000,1559865600,1559779200,1559692800,1559606400,1559520000,1559433600,1559347200,1559260800,1559174400,1559088000,1559001600,1558915200,1558828800,1558742400,1558656000,1558569600,1558483200,1558396800,1558310400,1558224000,1558137600
2,https://t.me/MaximumPump,Maximum Pump,ADX,Binance,2019-04-25T19:15:21,2019-04-25 19:15:21+00:00,1556219721,1556150400,1556064000,1555977600,1555891200,1555804800,1555718400,1555632000,1555545600,1555459200,1555372800,1555286400,1555200000,1555113600,1555027200,1554940800,1554854400,1554768000,1554681600,1554595200,1554508800,1554422400,1554336000,1554249600,1554163200,1554076800,1553990400,1553904000,1553817600,1553731200,1553644800,1553558400,1553472000,1553385600,1553299200,1553212800,1553126400,1553040000,1552953600,1552867200,1552780800,1552694400,1552608000,1552521600,1552435200,1552348800,1552262400,1552176000,1552089600,1552003200
3,https://t.me/bitcoinpumpgroup,Bitcoin Pump Group,WAN,Binance,2019-08-30T16:41:30Z,2019-08-30 16:41:30+00:00,1567183290,1567123200,1567036800,1566950400,1566864000,1566777600,1566691200,1566604800,1566518400,1566432000,1566345600,1566259200,1566172800,1566086400,1566000000,1565913600,1565827200,1565740800,1565654400,1565568000,1565481600,1565395200,1565308800,1565222400,1565136000,1565049600,1564963200,1564876800,1564790400,1564704000,1564617600,1564531200,1564444800,1564358400,1564272000,1564185600,1564099200,1564012800,1563926400,1563840000,1563753600,1563667200,1563580800,1563494400,1563408000,1563321600,1563235200,1563148800,1563062400,1562976000
4,https://t.me/bitcoinprofitcoach,Bitcoin Profit Coach,QSP,Binance,2019-08-29T19:46:43Z,2019-08-29 19:46:43+00:00,1567108003,1567036800,1566950400,1566864000,1566777600,1566691200,1566604800,1566518400,1566432000,1566345600,1566259200,1566172800,1566086400,1566000000,1565913600,1565827200,1565740800,1565654400,1565568000,1565481600,1565395200,1565308800,1565222400,1565136000,1565049600,1564963200,1564876800,1564790400,1564704000,1564617600,1564531200,1564444800,1564358400,1564272000,1564185600,1564099200,1564012800,1563926400,1563840000,1563753600,1563667200,1563580800,1563494400,1563408000,1563321600,1563235200,

### Test an Processed Satoshi dataset before Merging

In [11]:
s = pd.read_pickle("../../Coin_Data/Hourly/Processed_Data/Sat_Hourly_ADX_at_Bittrex.pkl")

In [12]:
s.head()

,time,close,high,low,open,volumefrom,volumeto,exchange,date,date_utc
0,1519488000,15020.0,15060.0,14760.0,14950.0,5.610200e+11,83410000.0,Bittrex,2018-02-24 08:00:00,2018-02-24 16:00:00+00:00
1,1519491600,15020.0,15080.0,14980.0,15020.0,7.146790e+11,107200000.0,Bittrex,2018-02-24 09:00:00,2018-02-24 17:00:00+00:00
2,1519495200,15110.0,15300.0,14980.0,15020.0,1.661654e+12,251700000.0,Bittrex,2018-02-24 10:00:00,2018-02-24 18:00:00+00:00
3,1519498800,14930.0,15190.0,14900.0,15110.0,1.723438e+12,258100000.0,Bittrex,2018-02-24 11:00:00,2018-02-24 19:00:00+00:00
4,1519502400,14890.0,14930.0,14760.0,14930.0,6.730940e+11,99810000.0,Bittrex,2018-02-24 12:00:00,2018-02-24 20:00:00+00:00


In [13]:
fin_vars = ['open','high','low','close','volumefrom','volumeto','date','date_utc']

### Step 2: Loop throug the processed social data file and merge based on timestemp
... We do not need to join by the coinID nor coin name. 

Just the __timestamps__ and __EXCHANGE__ are good enough. 
Since we want to retrieve the values for all available timestamps.

## HOURLY

In [47]:
dir_prefix = "../../Coin_Data/Hourly/Processed_Data/"
ohlc_error = []
x_final = pd.DataFrame()
t_temp = pd.DataFrame()
for filename in os.listdir(dir_prefix):
    if 'pkl' in filename:
        _,_,coin_name,_,exchange = filename.split('_')
        exchange = exchange[:-4]
        try: 
            print("Processing file...",filename)
            l = p.copy()
            for d in range(num_hours+1):
                if 'timestamp_h'+str(d) not in l.columns: 
                    print('Info. on hour_{} not in the left table!'.format(d))
                    continue
                x = pd.read_pickle(dir_prefix+filename)
                x.columns = [c+'_h'+str(d) if c != 'time' else 'time' for c in x.columns]
                #Merge the data based on time and exchange
              
                l = pd.merge(left=l,left_on=['timestamp_h'+str(d),'exchange']
                             ,right=x, right_on=["time","exchange_h"+str(d)]
                             ,how='inner')
#                 print(l.shape)
                l = l.drop(columns=['time',"exchange_h"+str(d)])
            #Save the file in the processed_directory
            l['coin_name'] = coin_name
            #Append all the data on top of each other for processing; 
            x_final = x_final.append(l)
        except: 
            e = sys.exc_info()[0]
            print( "<p>Error: %s</p>" % e )
            ohlc_error.append(filename)
            continue

Processing file... Sat_Hourly_SALT_at_Binance.pkl
Processing file... Sat_Hourly_POA_at_Binance.pkl
Processing file... Sat_Hourly_PINK_at_Bittrex.pkl
Processing file... Sat_Hourly_MDA_at_Binance.pkl
Processing file... Sat_Hourly_NLG_at_Bittrex.pkl
Processing file... Sat_Hourly_ENG_at_Binance.pkl
Processing file... Sat_Hourly_STORJ_at_Binance.pkl
Processing file... Sat_Hourly_EXCL_at_Bittrex.pkl
Processing file... Sat_Hourly_ING_at_Yobit.pkl
Processing file... Sat_Hourly_INS_at_Binance.pkl
Processing file... Sat_Hourly_DENT_at_Binance.pkl
Processing file... Sat_Hourly_ARDR_at_Binance.pkl
Processing file... Sat_Hourly_BTS_at_Binance.pkl
Processing file... Sat_Hourly_BOXX_at_Bittrex.pkl
Processing file... Sat_Hourly_BNT_at_Binance.pkl
Processing file... Sat_Hourly_BNT_at_Yobit.pkl
Processing file... Sat_Hourly_IOTX_at_Bittrex.pkl
Processing file... Sat_Hourly_MANA_at_Binance.pkl
Processing file... Sat_Hourly_UKG_at_Yobit.pkl
Processing file... Sat_Hourly_ETC_at_Yobit.pkl
Processing file...

Processing file... Sat_Hourly_BAT_at_Yobit.pkl
Processing file... Sat_Hourly_BCH_at_Yobit.pkl
Processing file... Sat_Hourly_AE_at_Yobit.pkl
Processing file... Sat_Hourly_QLC_at_Binance.pkl
Processing file... Sat_Hourly_TEC_at_Yobit.pkl
Processing file... Sat_Hourly_TNT_at_Binance.pkl
Processing file... Sat_Hourly_NEO_at_Binance.pkl
Processing file... Sat_Hourly_QSP_at_Binance.pkl
Processing file... Sat_Hourly_STORJ_at_Yobit.pkl
Processing file... Sat_Hourly_OST_at_Binance.pkl
Processing file... Sat_Hourly_STORM_at_Yobit.pkl
Processing file... Sat_Hourly_POLL_at_Yobit.pkl
Processing file... Sat_Hourly_SNT_at_Yobit.pkl
Processing file... Sat_Hourly_THETA_at_Binance.pkl
Processing file... Sat_Hourly_RLC_at_Bittrex.pkl
Processing file... Sat_Hourly_ICX_at_Binance.pkl
Processing file... Sat_Hourly_TKN_at_Yobit.pkl
Processing file... Sat_Hourly_NXS_at_Bittrex.pkl
Processing file... Sat_Hourly_NAV_at_Binance.pkl
Processing file... Sat_Hourly_LRC_at_Binance.pkl
Processing file... Sat_Hourly_SN

In [48]:
print(ohlc_error)
print(l.shape)

[]
(37, 449)


#We might have to check whether there is a non-zero data somewhere....

In [49]:
x_final = x_final.reset_index(drop=True)
x_final = x_final.sort_values(['timestamp','exchange'])
x_final = x_final.drop_duplicates()

In [50]:
x_final.filter(regex="open").head()

,open_h0,open_h1,open_h2,open_h3,open_h4,open_h5,open_h6,open_h7,open_h8,open_h9,open_h10,open_h11,open_h12,open_h13,open_h14,open_h15,open_h16,open_h17,open_h18,open_h19,open_h20,open_h21,open_h22,open_h23,open_h24,open_h25,open_h26,open_h27,open_h28,open_h29,open_h30,open_h31,open_h32,open_h33,open_h34,open_h35,open_h36,open_h37,open_h38,open_h39,open_h40,open_h41,open_h42,open_h43,open_h44,open_h45,open_h46,open_h47,open_h48
1710,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3347,33000.0,34000.0,34000.0,35000.0,36000.0,37000.0,39000.0,39000.0,40000.0,40000.0,39000.0,47000.0,47000.0,44000.0,46000.0,45000.0,46000.0,46000.0,30000.0,45000.0,45000.0,47000.0,47000.0,47000.0,46000.0,46000.0,49570.0,50000.0,53000.0,51000.0,54000.0,47000.0,50000.0,50000.0,50000.0,50000.0,48000.0,51000.0,52000.0,52000.0,52000.0,53000.0,54000.0,55000.0,54000.0,55000.0,55940.0,51000.0,53000.0
3844,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4019,231400.0,235000.0,244500.0,258800.0,262000.0,261500.0,261100.0,260800.0,256100.0,260100.0,274000.0,272700.0,276100.0,276000.0,262100.0,254900.0,248000.0,250300.0,251700.0,251300.0,248200.0,248000.0,250300.0,243000.0,253000.0,242700.0,244100.0,239000.0,264700.0,250000.0,229800.0,225400.0,220000.0,221000.0,215200.0,212700.0,218300.0,223800.0,188000.0,188700.0,199900.0,202100.0,197600.0,200000.0,202700.0,201600.0,209300.0,208100.0,205900.0
5371,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


__!!__ Notice that we have only __rougly half__ the number of obs. as the social media only case, 
but the additional constraints of exhange explains that (Few coins listed at each exchange).

*** Now we have the time series data up to 1 month in the past!
*** So now we can start preparing the data and sort the categories together?

In [51]:
print(x_final.shape)
x_final.head()

(43506, 449)


,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp,timestamp_h0,timestamp_h1,timestamp_h2,timestamp_h3,timestamp_h4,timestamp_h5,timestamp_h6,timestamp_h7,timestamp_h8,timestamp_h9,timestamp_h10,timestamp_h11,timestamp_h12,timestamp_h13,timestamp_h14,timestamp_h15,timestamp_h16,timestamp_h17,timestamp_h18,timestamp_h19,timestamp_h20,timestamp_h21,timestamp_h22,timestamp_h23,timestamp_h24,timestamp_h25,timestamp_h26,timestamp_h27,timestamp_h28,timestamp_h29,timestamp_h30,timestamp_h31,timestamp_h32,timestamp_h33,timestamp_h34,timestamp_h35,timestamp_h36,timestamp_h37,timestamp_h38,timestamp_h39,timestamp_h40,timestamp_h41,timestamp_h42,timestamp_h43,timestamp_h44,timestamp_h45,timestamp_h46,timestamp_h47,timestamp_h48,close_h0,high_h0,low_h0,open_h0,volumefrom_h0,volumeto_h0,date_h0,date_utc_h0,close_h1,high_h1,low_h1,open_h1,volumefrom_h1,volumeto_h1,date_h1,date_utc_h1,close_h2,high_h2,low_h2,open_h2,volumefrom_h2,volumeto_h2,date_h2,date_utc_h2,close_h3,high_h3,low_h3,open_h3,volumefrom_h3,volumeto_h3,date_h3,date_utc_h3,close_h4,high_h4,low_h4,open_h4,volumefrom_h4,volumeto_h4,date_h4,date_utc_h4,close_h5,high_h5,low_h5,open_h5,volumefrom_h5,volumeto_h5,date_h5,date_utc_h5,close_h6,high_h6,low_h6,open_h6,volumefrom_h6,volumeto_h6,date_h6,date_utc_h6,close_h7,high_h7,low_h7,open_h7,volumefrom_h7,volumeto_h7,date_h7,date_utc_h7,close_h8,high_h8,low_h8,open_h8,volumefrom_h8,volumeto_h8,date_h8,date_utc_h8,close_h9,high_h9,low_h9,open_h9,volumefrom_h9,volumeto_h9,date_h9,date_utc_h9,close_h10,high_h10,low_h10,open_h10,volumefrom_h10,volumeto_h10,date_h10,date_utc_h10,close_h11,high_h11,low_h11,open_h11,volumefrom_h11,volumeto_h11,date_h11,date_utc_h11,close_h12,high_h12,low_h12,open_h12,volumefrom_h12,volumeto_h12,date_h12,date_utc_h12,close_h13,high_h13,low_h13,open_h13,volumefrom_h13,volumeto_h13,date_h13,date_utc_h13,close_h14,high_h14,low_h14,open_h14,volumefrom_h14,volumeto_h14,date_h14,date_utc_h14,close_h15,high_h15,low_h15,open_h15,volumefrom_h15,volumeto_h15,date_h15,date_utc_h15,close_h16,high_h16,low_h16,open_h16,volumefrom_h16,volumeto_h16,date_h16,date_utc_h16,close_h17,high_h17,low_h17,open_h17,volumefrom_h17,volumeto_h17,date_h17,date_utc_h17,close_h18,high_h18,low_h18,open_h18,volumefrom_h18,volumeto_h18,date_h18,date_utc_h18,close_h19,high_h19,low_h19,open_h19,volumefrom_h19,volumeto_h19,date_h19,date_utc_h19,close_h20,high_h20,low_h20,open_h20,volumefrom_h20,volumeto_h20,date_h20,date_utc_h20,close_h21,high_h21,low_h21,open_h21,volumefrom_h21,volumeto_h21,date_h21,date_utc_h21,close_h22,high_h22,low_h22,open_h22,volumefrom_h22,volumeto_h22,date_h22,date_utc_h22,close_h23,high_h23,low_h23,open_h23,volumefrom_h23,volumeto_h23,date_h23,date_utc_h23,close_h24,high_h24,low_h24,open_h24,volumefrom_h24,volumeto_h24,date_h24,date_utc_h24,close_h25,high_h25,low_h25,open_h25,volumefrom_h25,volumeto_h25,date_h25,date_utc_h25,close_h26,high_h26,low_h26,open_h26,volumefrom_h26,volumeto_h26,date_h26,date_utc_h26,close_h27,high_h27,low_h27,open_h27,volumefrom_h27,volumeto_h27,date_h27,date_utc_h27,close_h28,high_h28,low_h28,open_h28,volumefrom_h28,volumeto_h28,date_h28,date_utc_h28,close_h29,high_h29,low_h29,open_h29,volumefrom_h29,volumeto_h29,date_h29,date_utc_h29,close_h30,high_h30,low_h30,open_h30,volumefrom_h30,volumeto_h30,date_h30,date_utc_h30,close_h31,high_h31,low_h31,open_h31,volumefrom_h31,volumeto_h31,date_h31,date_utc_h31,close_h32,high_h32,low_h32,open_h32,volumefrom_h32,volumeto_h32,date_h32,date_utc_h32,close_h33,high_h33,low_h33,open_h33,volumefrom_h33,volumeto_h33,date_h33,date_utc_h33,close_h34,high_h34,low_h34,open_h34,volumefrom_h34,volumeto_h34,date_h34,date_utc_h34,close_h35,high_h35,low_h35,open_h35,volumefrom_h35,volumeto_h35,date_h35,date_utc_h35,close_h36,high_h36,low_h36,open_h36,volumefrom_h36,volumeto_h36,date_h36,date_utc_h36,close_h37,high_h37,low_h37,open_h37,volumefrom_h37,volumeto_h37,date_h37,date_utc_h37,close_h38,high_h38,low_h38,open_h38,volumefrom_h38,volu

In [52]:
col_dict = dict()
for f in ['open','high','low','close','volumefrom','volumeto']:
    col_dict[f] = [f+"_h"+str(h) for h in range(num_hours + 1)]

In [53]:
# col_dict

In [54]:
core = ['channelLink','channelTitle','currency','exchange',
        'timestamp','signalTime','pumptime_edited','coin_name']
col_dict['core'] = core

In [55]:
col_dict.keys()

dict_keys(['open', 'high', 'low', 'close', 'volumefrom', 'volumeto', 'core'])

In [56]:
ohlcv_vars = []
for k,v in col_dict.items():
#     if k not in ['core','volumefrom']:
        ohlcv_vars+=col_dict[k]

## Make sure all the importmnat columns are preserved

In [57]:
len(ohlcv_vars)

302

In [58]:
orig_cols = x_final.columns
for i in orig_cols:
    if i not in ohlcv_vars:
        print(i)

timestamp_h0
timestamp_h1
timestamp_h2
timestamp_h3
timestamp_h4
timestamp_h5
timestamp_h6
timestamp_h7
timestamp_h8
timestamp_h9
timestamp_h10
timestamp_h11
timestamp_h12
timestamp_h13
timestamp_h14
timestamp_h15
timestamp_h16
timestamp_h17
timestamp_h18
timestamp_h19
timestamp_h20
timestamp_h21
timestamp_h22
timestamp_h23
timestamp_h24
timestamp_h25
timestamp_h26
timestamp_h27
timestamp_h28
timestamp_h29
timestamp_h30
timestamp_h31
timestamp_h32
timestamp_h33
timestamp_h34
timestamp_h35
timestamp_h36
timestamp_h37
timestamp_h38
timestamp_h39
timestamp_h40
timestamp_h41
timestamp_h42
timestamp_h43
timestamp_h44
timestamp_h45
timestamp_h46
timestamp_h47
timestamp_h48
date_h0
date_utc_h0
date_h1
date_utc_h1
date_h2
date_utc_h2
date_h3
date_utc_h3
date_h4
date_utc_h4
date_h5
date_utc_h5
date_h6
date_utc_h6
date_h7
date_utc_h7
date_h8
date_utc_h8
date_h9
date_utc_h9
date_h10
date_utc_h10
date_h11
date_utc_h11
date_h12
date_utc_h12
date_h13
date_utc_h13
date_h14
date_utc_h14
date_h15
date_

In [59]:
x_final = x_final[ohlcv_vars]
x_final = x_final.reset_index(drop=True)

In [60]:
x_final.head(2)

,open_h0,open_h1,open_h2,open_h3,open_h4,open_h5,open_h6,open_h7,open_h8,open_h9,open_h10,open_h11,open_h12,open_h13,open_h14,open_h15,open_h16,open_h17,open_h18,open_h19,open_h20,open_h21,open_h22,open_h23,open_h24,open_h25,open_h26,open_h27,open_h28,open_h29,open_h30,open_h31,open_h32,open_h33,open_h34,open_h35,open_h36,open_h37,open_h38,open_h39,open_h40,open_h41,open_h42,open_h43,open_h44,open_h45,open_h46,open_h47,open_h48,high_h0,high_h1,high_h2,high_h3,high_h4,high_h5,high_h6,high_h7,high_h8,high_h9,high_h10,high_h11,high_h12,high_h13,high_h14,high_h15,high_h16,high_h17,high_h18,high_h19,high_h20,high_h21,high_h22,high_h23,high_h24,high_h25,high_h26,high_h27,high_h28,high_h29,high_h30,high_h31,high_h32,high_h33,high_h34,high_h35,high_h36,high_h37,high_h38,high_h39,high_h40,high_h41,high_h42,high_h43,high_h44,high_h45,high_h46,high_h47,high_h48,low_h0,low_h1,low_h2,low_h3,low_h4,low_h5,low_h6,low_h7,low_h8,low_h9,low_h10,low_h11,low_h12,low_h13,low_h14,low_h15,low_h16,low_h17,low_h18,low_h19,low_h20,low_h21,low_h22,low_h23,low_h24,low_h25,low_h26,low_h27,low_h28,low_h29,low_h30,low_h31,low_h32,low_h33,low_h34,low_h35,low_h36,low_h37,low_h38,low_h39,low_h40,low_h41,low_h42,low_h43,low_h44,low_h45,low_h46,low_h47,low_h48,close_h0,close_h1,close_h2,close_h3,close_h4,close_h5,close_h6,close_h7,close_h8,close_h9,close_h10,close_h11,close_h12,close_h13,close_h14,close_h15,close_h16,close_h17,close_h18,close_h19,close_h20,close_h21,close_h22,close_h23,close_h24,close_h25,close_h26,close_h27,close_h28,close_h29,close_h30,close_h31,close_h32,close_h33,close_h34,close_h35,close_h36,close_h37,close_h38,close_h39,close_h40,close_h41,close_h42,close_h43,close_h44,close_h45,close_h46,close_h47,close_h48,volumefrom_h0,volumefrom_h1,volumefrom_h2,volumefrom_h3,volumefrom_h4,volumefrom_h5,volumefrom_h6,volumefrom_h7,volumefrom_h8,volumefrom_h9,volumefrom_h10,volumefrom_h11,volumefrom_h12,volumefrom_h13,volumefrom_h14,volumefrom_h15,volumefrom_h16,volumefrom_h17,volumefrom_h18,volumefrom_h19,volumefrom_h20,volumefrom_h21,volumefrom_h22,volumefrom_h23,volumefrom_h24,volumefrom_h25,volumefrom_h26,volumefrom_h27,volumefrom_h28,volumefrom_h29,volumefrom_h30,volumefrom_h31,volumefrom_h32,volumefrom_h33,volumefrom_h34,volumefrom_h35,volumefrom_h36,volumefrom_h37,volumefrom_h38,volumefrom_h39,volumefrom_h40,volumefrom_h41,volumefrom_h42,volumefrom_h43,volumefrom_h44,volumefrom_h45,volumefrom_h46,volumefrom_h47,volumefrom_h48,volumeto_h0,volumeto_h1,volumeto_h2,volumeto_h3,volumeto_h4,volumeto_h5,volumeto_h6,volumeto_h7,volumeto_h8,volumeto_h9,volumeto_h10,volumeto_h11,volumeto_h12,volumeto_h13,volumeto_h14,volumeto_h15,volumeto_h16,volumeto_h17,volumeto_h18,volumeto_h19,volumeto_h20,volumeto_h21,volumeto_h22,volumeto_h23,volumeto_h24,volumeto_h25,volumeto_h26,volumeto_h27,volumeto_h28,volumeto_h29,volumeto_h30,volumeto_h31,volumeto_h32,volumeto_h33,volumeto_h34,volumeto_h35,volumeto_h36,volumeto_h37,volumeto_h38,volumeto_h39,volumeto_h40,volumeto_h41,volumeto_h42,volumeto_h43,volumeto_h44,volumeto_h45,volumeto_h46,volumeto_h47,volumeto_h48,channelLink,channelTitle,currency,exchange,timestamp,signalTime,pumptime_edited,coin_name
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0

In [61]:
x_final.to_pickle("ohlcv_data_training_h.pkl")

### Also save the column names just to be sure


In [62]:
f = open("ohlcv_dict_h.pkl","wb")
pickle.dump(col_dict,f)
f.close()

g = open("ohlcv_features_h.pkl","wb")
pickle.dump(ohlcv_vars,g)
g.close()

In [63]:
ohlcv_vars

['open_h0',
 'open_h1',
 'open_h2',
 'open_h3',
 'open_h4',
 'open_h5',
 'open_h6',
 'open_h7',
 'open_h8',
 'open_h9',
 'open_h10',
 'open_h11',
 'open_h12',
 'open_h13',
 'open_h14',
 'open_h15',
 'open_h16',
 'open_h17',
 'open_h18',
 'open_h19',
 'open_h20',
 'open_h21',
 'open_h22',
 'open_h23',
 'open_h24',
 'open_h25',
 'open_h26',
 'open_h27',
 'open_h28',
 'open_h29',
 'open_h30',
 'open_h31',
 'open_h32',
 'open_h33',
 'open_h34',
 'open_h35',
 'open_h36',
 'open_h37',
 'open_h38',
 'open_h39',
 'open_h40',
 'open_h41',
 'open_h42',
 'open_h43',
 'open_h44',
 'open_h45',
 'open_h46',
 'open_h47',
 'open_h48',
 'high_h0',
 'high_h1',
 'high_h2',
 'high_h3',
 'high_h4',
 'high_h5',
 'high_h6',
 'high_h7',
 'high_h8',
 'high_h9',
 'high_h10',
 'high_h11',
 'high_h12',
 'high_h13',
 'high_h14',
 'high_h15',
 'high_h16',
 'high_h17',
 'high_h18',
 'high_h19',
 'high_h20',
 'high_h21',
 'high_h22',
 'high_h23',
 'high_h24',
 'high_h25',
 'high_h26',
 'high_h27',
 'high_h28',
 'high